In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import ee

from os import listdir
from os.path import join
import os

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
startDate = pd.to_datetime('2000-08-01')
endDate = pd.to_datetime('2023-08-01')
dates_list = pd.date_range(start='2000-08-01', freq='AS-AUG', periods=24)

lat, lon = 45,60
point = ee.Geometry.Point([lon, lat]).buffer(distance=5000)

In [4]:
IMGS = list()
for date in dates_list:
  MOD = ee.ImageCollection("MODIS/061/MOD09GA")\
                .filterDate(start = date)\
                .filterBounds(point)
  IMGS.append(ee.Image(MOD.first()))

# Mapping


In [ ]:
!pip install geemap

In [8]:
import geemap

In [9]:
bands = ['sur_refl_b02', 'sur_refl_b04', 'sur_refl_b03']

In [10]:
map = geemap.Map(center=[lat, lon], zoom=8)
roi = ee.Geometry.Rectangle([57, 43,62, 47])
pars = {'min': -100.0,
  'max': 8000.0,}
map.add_ee_layer(IMGS[0].select(bands).clip(roi), pars)
map

Map(center=[45, 60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

# GIF

In [ ]:
col = geemap.create_timeseries(
        ee.ImageCollection(IMGS),
        '2000-08-01',
        '2023-08-01',
        region=roi,
        bands=bands,
    )
col

In [ ]:
col = col.select(bands).map(
        lambda img: img.visualize(**pars).set(
            {
                "system:time_start": img.get("system:time_start"),
                "system:date": img.get("system:date"),
            }
        )
    )

In [ ]:
video_args = {}
video_args["dimensions"] = 768
video_args["region"] = roi
video_args["framesPerSecond"] = 30
video_args["crs"] = "EPSG:3857"
video_args["min"] = 0
video_args["max"] = 255
video_args["bands"] = ["vis-red", "vis-green", "vis-blue"]

In [ ]:
out_dir=''
count = col.size().getInfo()
basename = 'MOD'
names = [
    os.path.join(
        out_dir, f"{basename}_{str(i+1).zfill(int(len(str(count))))}.jpg"
    )
    for i in range(count)
]
geemap.get_image_collection_thumbnails(
            col,
            './MOD',
            vis_params={
                "min": 0,
                "max": 255,
                "bands": video_args["bands"],
            },
            dimensions=768,
            names=names,
        )

geemap.make_gif(
    ['./MOD/'+x for x in names],
    'MOD.gif',
    fps=2,
    mp4=False,
    clean_up=True,
)

geemap.add_text_to_gif(
  'MOD.gif',
  'MOD.gif',
  text_sequence=[x.strftime('%Y-%m-%d') for x in dates_list],
  font_type='monospace',
  font_size=24,
  font_color='white',
  duration=1000 / 3,

)

Total number of images: 24

The specified font type could not be found on your system. Using the default font instead.
